### Decorators Part 2 - lekce

Dekorátory s parametry jsou function call, @timed - nemá function call, @wraps(fn) nebo @lru_cache(maxsize=256) ..to mají trochu jinak

V podstatě abychom docílii decoratoru s parametry - tvoříme nested closure?  3 funkce v sobě

Takže pokud máme decorator s parametry jedná se o funkci, která vytváří decorator, neboli decorator factory.

In [221]:
def timed(fn):
    from time import perf_counter
    
    def inner(*args,**kwargs):
        start = perf_counter()
        result = fn(*args,**kwargs)
        end = perf_counter()
        elapsed = end - start
        print(f"run time: {elapsed: .6f}s")
        return result
    return inner
#pouze opakuje timed funkci

In [218]:
def cal_fib_recurse(n):
    return 1 if n < 3 else cal_fib_recurse(n-2) + cal_fib_recurse(n-1)

In [220]:
def fib(n):
    return cal_fib_recurse(n)
#pouze obaluji prvotní funkc

In [222]:
fib = timed(fib) #old school dekorace - pro názornost abychom se dostali k parametrům

In [223]:
fib(10)

run time:  0.000016s


55

Nyní chceme víckrát, prozatím hard_coded pomocí for loopu a čísla 10

In [224]:
def timed(fn):
    from time import perf_counter
       
    def inner(*args,**kwargs):
        total_elapsed = 0
        for i in range(10):
            start = perf_counter()
            result = fn(*args,**kwargs)
            end = perf_counter()
            total_elapsed += (end-start)
            
        avg_run_time = total_elapsed / 10
        print(f"AVG run time: {avg_run_time: .6f}s")
        return result
    return inner
#v podstatě jsem upravil funkci tak aby se opakovala 10x
#a z průměrovala výsledek

In [225]:
def fib2(n):
    return cal_fib_recurse(n)

In [226]:
fib2 = timed(fib2)

In [227]:
fib2(28)

AVG run time:  0.100342s


317811

Nyní máme tedy průměrný výpočet pro 10 průběhů. Nyní ale potřebujeme hard_coded variantu převést na optional tzn. parametr.

In [249]:
def timed2(fn, reps):
    from time import perf_counter
       
    def inner(*args,**kwargs):
        total_elapsed = 0
        for i in range(reps):
            start = perf_counter()
            result = fn(*args,**kwargs)
            end = perf_counter()
            total_elapsed += (end-start)
            
        avg_run_time = total_elapsed / reps
        print(f"AVG run time: {avg_run_time: .6f}s, \nrepetition number: {reps}")
        return result
    return inner
#a nyní máme náš skvělý parametr

In [257]:
def fib3(n):
    return cal_fib_recurse(n)

In [258]:
fib3 = timed2(fib3, 7)

In [259]:
fib3(8)

AVG run time:  0.000014s, 
repetition number: 7


21

Nyní máme tedy funkční dekorátor s parametrem. Ale nefunguje v podobě @ - nemůžeme převést na decorátor takto, musíme přetvořit na decorator factory.

In [261]:
def dec(fn):
    print("running dec")
    
    def inner(*args,**kwargs):
        print("running inner")
        return fn(*args,**kwargs)
    return inner
#basic decorátor

In [262]:
@dec
def my_func():
    print("running my_func")

running dec


In [263]:
def my_func():
    print("running my_func")
my_func = dec(my_func)
#stejný zápis

running dec


V podstatě tiskne running dec - protože to probíhá automaticky.

In [265]:
def dec_factory():
    print("running dec_factory")
    
    def dec(fn):
        print("running dec")
    
        def inner(*args,**kwargs):
            print("running inner")
            return fn(*args,**kwargs)
        
        return inner
    return dec
#mám obalený dekorátor v dec_factory

In [266]:
dec = dec_factory() 

running dec_factory


In [267]:
def my_func():
    print("running my_func")

In [268]:
my_func = dec(my_func)

running dec


In [269]:
@dec
def my_func():
    print("running my_func")

running dec


dec = dec_factory()  -> proto když chci použít faktory jako dekorátor - musím mít ()

In [270]:
@dec_factory() #a proto máme dekorátor s ()
def my_func():
    print("running my_func")
#a tudíž si mohou předávat parametry

running dec_factory
running dec


In [271]:
def my_func():
    print("running my_func")
my_func = dec_factory()(my_func) #stejný zápis
#v podstatě volá funkci co volá funkci

running dec_factory
running dec


In [273]:
def dec_factory(a,b):
    print("running dec_factory")
    
    def dec(fn):
        print("running dec")
    
        def inner(*args,**kwargs):
            print("running inner")
            print(f"a={a}, b={b}")
            return fn(*args,**kwargs)
        
        return inner
    return dec
#nyní přidáváme parametry

In [277]:
dec = dec_factory(10,20) #továrna maká

running dec_factory


In [278]:
@dec
def my_func():
    print("running my_func") #prostřední dekorátor

running dec


In [279]:
my_func() #volání funkce - předáváme na inner - tiskneme a,b

running inner
a=10, b=20
running my_func


In [280]:
@dec_factory(100,200)
def my_func():
    print("running my_func") 

running dec_factory
running dec


In [281]:
my_func()

running inner
a=100, b=200
running my_func


A takto to již máme přesněji s parametry. Nyní upravíme timed zpátky.

In [285]:
def dec_factory(reps):
    def timed3(fn):
        from time import perf_counter

        def inner(*args,**kwargs):
            total_elapsed = 0
            for i in range(reps):
                start = perf_counter()
                result = fn(*args,**kwargs)
                end = perf_counter()
                total_elapsed += (end-start)

            avg_run_time = total_elapsed / reps
            print(f"AVG run time: {avg_run_time: .6f}s, \nrepetition number: {reps}")
            return result
        return inner
    return timed3

In [286]:
@dec_factory(5)
def fib3(n):
    return cal_fib_recurse(n)

In [287]:
fib3(5)

AVG run time:  0.000007s, 
repetition number: 5


5

A nyní máme dekorátor funkční i s parametrem. V podstatě se to oblaí o funkci dále.

Ideální je lepší pojmenování timed - OUTER - INNER .nebo TIME - DEC - INNER